In [ ]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None
frame_changed = False

async def generate_frames():
    global latest_frame
    global frame_changed
    global delay
    c = 0
    # Variable for frame interval (in seconds) to achieve desired FPS
    frame_interval = 1 / 4
    start_time = time.time()
    num_frames = 0
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        # print(f'total {num_frames}')
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()

        if True:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)
            c += 1
            num_frames += 1

            latest_frame = buffer.tobytes()  # Update the latest frame
            frame_changed = True
            

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)

        frame_time = time.time() - frame_start_time
        # Calculate the delay required to achieve desired FPS
        delay = frame_interval - frame_time

        

        try:
            # Introduce the delay
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global latest_frame
        global frame_changed
        num_frames = 0
        all_frames = 0
        while True:
            # all_frames +=1
            # if frame_changed==True:
            #     num_frames += 1
                

            if frame_changed==True:  # Check if there's a latest frame available
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
                    frame_changed = False
            else:
                # print(delay)
                await asyncio.sleep(delay)  # If no latest frame available, wait for a short time
        # print(f'total {num_frames}')


    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())
    uvicorn.run(app, host="127.0.0.1", port=8000)



In [2]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None
frame_changed = False

async def generate_frames():
    global latest_frame
    global frame_changed
    global delay
    c = 0
    # Variable for frame interval (in seconds) to achieve desired FPS
    frame_interval = 1 / 4
    start_time = time.time()
    num_frames = 0
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        # print(f'total {num_frames}')
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()

        if ret:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)
            c += 1
            num_frames += 1

            latest_frame = buffer.tobytes()  # Update the latest frame
            frame_changed = True
        else:
            frame_changed =False
@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global latest_frame
        global frame_changed
        num_frames = 0
        all_frames = 0
        while True:
            # all_frames +=1
            # if frame_changed==True:
            #     num_frames += 1
                

            if frame_changed==True:  # Check if there's a latest frame available
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )

            else:
                # print(delay)
                await asyncio.sleep(0.01)  # If no latest frame available, wait for a short time
        # print(f'total {num_frames}')


    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())
    uvicorn.run(app, host="127.0.0.1", port=8000)



INFO:     Started server process [8640]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [8640]


In [2]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import numpy as np
import asyncio
import threading
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

# Shared variables to store the latest frame and flag indicating frame change
latest_frame = None
frame_changed = False

def generate_frames():
    global latest_frame
    global frame_changed
    num_frames=0

    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()

        if ret:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)
            num_frames += 1

            latest_frame = buffer.tobytes()  # Update the latest frame
            frame_changed = True

            

def start_frame_generation():
    # Start the frame generation in a separate thread
    threading.Thread(target=generate_frames, daemon=True).start()

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global frame_changed

        while True:
            if frame_changed!=False:  # Check if there's a latest frame available
                yield (
                    b'--frame\r\n'
                    b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                )
                frame_changed = False
            else:
                await asyncio.sleep(0.02)  # If no latest frame available, wait for a short time

    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    start_frame_generation()  # Start frame generation in a separate thread
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [4792]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51424 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:51434 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:51435 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:51444 - "GET /video_feed HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4792]


In [ ]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
import threading
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None
frame_changed = False
stream = False
global latest_frame, frame_changed, delay, l, num_frames

async def generate_frames():

    global latest_frame, frame_changed, delay, l, num_frames
    # Variable for frame interval (in seconds) to achieve desired FPS
    frame_interval = 1 / 4
    start_time = time.time()
    
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    l = None
    num_frames = 0
    
    while True:
        # print(f'total {num_frames}')
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()
        l=num_frames

        if True:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)

            num_frames += 1


            latest_frame = buffer.tobytes()  # Update the latest frame

        print(l)

        

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)

        frame_time = time.time() - frame_start_time
        # Calculate the delay required to achieve desired FPS
        delay = frame_interval - frame_time

        

        try:
            # Introduce the delay
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break
@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global latest_frame

        while True:
            # all_frames +=1
            # if frame_changed==True:
            #     num_frames += 1
            print(f'num frames {num_frames}')
            print(f"l {l}")
            if num_frames > l:  # Check if there's a latest frame available
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
            else:
                # print(delay)
                await asyncio.sleep(0.01)  # If no latest frame available, wait for a short time
        # print(f'total {num_frames}')


    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())
    uvicorn.run(app, host="127.0.0.1", port=8000)



In [ ]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
import threading
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None
frame_changed = False
stream = False
global latest_frame, frame_changed, delay, l, num_frames

async def generate_frames():

    global latest_frame, frame_changed, delay, l, num_frames
    # Variable for frame interval (in seconds) to achieve desired FPS
    frame_interval = 1 / 4
    start_time = time.time()
    
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    l = None
    num_frames = 0
    
    while True:
        # print(f'total {num_frames}')
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()
        l=num_frames
        

        if True:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)

            num_frames += 1


            latest_frame = buffer.tobytes()  # Update the latest frame

        print(l)

        

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)

        frame_time = time.time() - frame_start_time
        # Calculate the delay required to achieve desired FPS
        delay = frame_interval - frame_time

        

        try:
            # Introduce the delay
            await asyncio.sleep(delay)
            l=num_frames
        except asyncio.CancelledError:
            break
@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global latest_frame
        lc =[0,]

        while True:
            # all_frames +=1
            # if frame_changed==True:
            #     num_frames += 1
            ll = lc[-1]
            if l==ll:
                lc.append(l)
            # print(f'num frames {num_frames}')
            
            # print(f"l {l}")
            if num_frames > l and l!=ll:  # Check if there's a latest frame available
                    lc.append(l)
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
            else:
                # print(delay)
                await asyncio.sleep(0.01)  # If no latest frame available, wait for a short time
        # print(f'total {num_frames}')


    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())
    uvicorn.run(app, host="127.0.0.1", port=8000)



In [ ]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
import threading
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None
frame_changed = False
stream = False
global latest_frame, frame_changed, delay, l, num_frames

async def generate_frames():

    global latest_frame, frame_changed, delay, l, num_frames
    # Variable for frame interval (in seconds) to achieve desired FPS
    frame_interval = 1 / 4
    start_time = time.time()
    
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    l = None
    num_frames = 0
    
    while True:
        # print(f'total {num_frames}')
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()
        l=num_frames
        

        if True:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)

            num_frames += 1


            latest_frame = buffer.tobytes()  # Update the latest frame

        print(l)

        

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)

        frame_time = time.time() - frame_start_time
        # Calculate the delay required to achieve desired FPS
        delay = frame_interval - frame_time

        

        try:
            # Introduce the delay
            await asyncio.sleep(delay)
            l=num_frames
        except asyncio.CancelledError:
            break

def perform_calculation():
    lc = [0,]
    while True:
        ll = lc[-1]
        if l == ll:
            lc.append(l)
        # print(f'num frames {num_frames}')
        # print(f"l {l}")
    
@app.get("/video_feed")
async def video_feed():
    # Create a separate thread for the calculation
    calc_thread = threading.Thread(target=perform_calculation)
    calc_thread.daemon = True  # Set as daemon thread so it exits when main thread exits
    calc_thread.start()

    async def stream_frame():
        global latest_frame
        lc =[0,]

        while True:
            # all_frames +=1
            # if frame_changed==True:
            #     num_frames += 1
            ll = lc[-1]
            if l==ll:
                lc.append(l)
            # print(f'num frames {num_frames}')
            
            # print(f"l {l}")
            if num_frames > l and l!=ll:  # Check if there's a latest frame available
                    lc.append(l)
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
            else:
                # print(delay)
                await asyncio.sleep(0.01)  # If no latest frame available, wait for a short time
        # print(f'total {num_frames}')


    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())
    uvicorn.run(app, host="127.0.0.1", port=8000)



In [ ]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
import threading
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None
frame_changed = False
ll=None
global latest_frame, frame_changed, delay, l, num_frames

async def generate_frames():

    global latest_frame, frame_changed, delay, l, num_frames
    # Variable for frame interval (in seconds) to achieve desired FPS
    frame_interval = 1 / 4
    start_time = time.time()
    
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    l = None
    num_frames = 0
    
    while True:
        # print(f'total {num_frames}')
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()
        l=num_frames
        

        if True:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)

            num_frames += 1


            latest_frame = buffer.tobytes()  # Update the latest frame

        # print(l)

        

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)

        frame_time = time.time() - frame_start_time
        # Calculate the delay required to achieve desired FPS
        delay = frame_interval - frame_time

        

        try:
            # Introduce the delay
            await asyncio.sleep(delay)
            l=num_frames
        except asyncio.CancelledError:
            break

def perform_calculation():
    global lc
    global ll
    lc = [0,]
    while True:
        
        ll = lc[-1]
        # print(f'calulate ll {ll}')
        if l == ll:
            lc.append(l)
        # print(f'num frames {num_frames}')
        # print(f"l {l}")
calc_thread = threading.Thread(target=perform_calculation)
calc_thread.daemon = True  # Set as daemon thread so it exits when main thread exits
calc_thread.start()
@app.get("/video_feed")
async def video_feed():
    # Create a separate thread for the calculation
    
    
    

    async def stream_frame():
        global lc
        while True:
            # print(f"l {l}")
            if num_frames > l and l!=ll:  # Check if there's a latest frame available
                    lc.append(l)
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
            else:
                # print(delay)
                await asyncio.sleep(0.005)  # If no latest frame available, wait for a short time
        # print(f'total {num_frames}')


    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())

    uvicorn.run(app, host="127.0.0.1", port=8000)



INFO:     Started server process [40252]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58928 - "GET /video_feed HTTP/1.1" 200 OK


In [ ]:
d = 0
c=0
for i in range(5):
    c+=1
    d=c
    print(d)

In [ ]:
l =[0,]
for i in range(10):
    l.append(i)

In [ ]:
l[-1]

In [ ]:
l

In [3]:
0.01/2

0.005

In [ ]:
# %%writefile server.py

from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import numpy as np
import datetime
import asyncio
from queue import Queue
import nest_asyncio
nest_asyncio.apply()
app = FastAPI()

# Shared variable to store the latest frame
latest_frame = None
frame_changed = False

async def generate_frames():
    global latest_frame
    global frame_changed
    global delay
    c = 0
    # Variable for frame interval (in seconds) to achieve desired FPS
    frame_interval = 1 / 4
    start_time = time.time()
    num_frames = 0
    url = "http://localhost:3000/video_feed"  # Update the URL if needed
    cap = cv2.VideoCapture(url)
    while True:
        # print(f'total {num_frames}')
        # Record start time of processing a frame
        frame_start_time = time.time()
        ret, frame = cap.read()

        if True:
            # Convert the frame to JPEG format for streaming
            _, buffer = cv2.imencode('.jpg', frame)
            c += 1
            num_frames += 1

            latest_frame = buffer.tobytes()  # Update the latest frame
            frame_changed = True
            

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        # If one second has elapsed, calculate and print FPS
        if elapsed_time >= 1:
            fps = num_frames / elapsed_time
            # print("FPS:", fps)

        frame_time = time.time() - frame_start_time
        # Calculate the delay required to achieve desired FPS
        delay = frame_interval - frame_time

        

        try:
            # Introduce the delay
            await asyncio.sleep(delay)
        except asyncio.CancelledError:
            break

@app.get("/video_feed")
async def video_feed():
    async def stream_frame():
        global latest_frame
        global frame_changed
        num_frames = 0
        all_frames = 0
        while True:

            if frame_changed==True:  # Check if there's a latest frame available
                    yield (
                        b'--frame\r\n'
                        b'Content-Type: image/jpeg\r\n\r\n' + latest_frame + b'\r\n'
                    )
                    frame_changed = False
            else:
                # print(delay)
                await asyncio.sleep(delay)  # If no latest frame available, wait for a short time
        # print(f'total {num_frames}')


    return StreamingResponse(stream_frame(), media_type="multipart/x-mixed-replace; boundary=frame")

async def main():
    # Start generating frames asynchronously
    asyncio.create_task(generate_frames())

if __name__ == "__main__":
    asyncio.run(main())
    uvicorn.run(app, host="127.0.0.1", port=8000)



INFO:     Started server process [84660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57525 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:57600 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:57599 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:57601 - "GET /video_feed HTTP/1.1" 200 OK
INFO:     127.0.0.1:57602 - "GET /video_feed HTTP/1.1" 200 OK
